<div id="colab_button">
  <h1>Quick tour</h1>
  <a target="_blank" href="https://colab.research.google.com/github/mithril-security/blindai-preview/blob/main/docs/docs/getting-started/quick-tour.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>
______________________________

In this notebook, we will explore how **BlindAI** can be used to **query popular AI models** while offering **strong privacy guarantees**. It does so by processing user data in a **SGX secure enclave**.

To let you discover BlindAI without having to install anything locally, we made [**a Google Colab**](https://colab.research.google.com/github/mithril-security/blindai-preview/blob/main/docs/docs/getting-started/quick-tour.ipynb) where you can directly follow this introduction tutorial with or without a Google account. However, everything has been explained and packaged for you to be able to run tests on your machine, if you wish to.

> If you're still wondering why you should use BlindAI, you can go check [our introduction](https://blindai-preview.mithrilsecurity.io/en/latest/docs/getting-started/why-blindai/) on that topic. We'll focus on a hands-on demo in this tutorial.


### The use case: Secure audio transcription

In this tutorial, we take the use case of a hospital that wants to automate transcriptions of consultations with patients. This would help doctors to make a record of everything covered in consultations which they can refer back to at any time, saving the need for them to take time out to write up lengthy notes.

There are lots of existing speech-to-text applications, but the hospital has to be very careful- this is highly sensitive and critical patient data. They need to be sure that their data cannot be accessed or tampered by any third party, whether that be the speech-to-text application creator, the application's Cloud provider or a malicious hacker targetting one of these... And that's where BlindAI comes in!

BlindAI offers secure audio transcription using the Whisper model, an automatic speech recognition (ASR) system trained on 680,000 hours of multilingual and multitask supervised data collected from the web. Data sent to any of the models available on BlindAI is kept confidential at all times- no one can access or tamper with the data. You can learn more about how user data is protecter [here](LIEN).


So with that said, let's take a look at how to set-up and use BlindAI.

## Pre-requisites
________________________________________________________

Before we begin, you will need to install the [blindai-preview](https://pypi.org/project/blindai-preview/) Python library and download a [test audio file](https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav) provided by University of Illinois at Chicago  as part of their Computer Science 101 course.

In [ ]:
# install blindai
!pip install blindai-preview

# Download our example audio file and save it as `taunt.wav`
!wget https://www2.cs.uic.edu/~i101/SoundFiles/taunt.wav -O taunt.wav

## Transcribing our audio file
______________________________________________

Speech-to-text operations are handled within the `Audio.transcribe` module of `BlindAI.api`.

To transcribe an audio file, we call the `trascribe` method and provide the path to our audio file as the `file` option. The Whisper model accepts a variety of audio file formats (m4a, mp3, mp4, mpeg, mpga, wav, webm).

Under the hood, BlindAI connects to our managed cloud solution which has an instance of the BlindAI server running and queries its pre-uploaded Whisper model before returning the transcribed text results as a string.

In [ ]:
transcript = blindai_preview.api.Audio.transcribe(
    file="taunt.wav"
)

Downloading...
From: https://drive.google.com/uc?id=1wqg1F0UkEdm3KB7n1BjfRLHnzKU2-G5S
To: /workspaces/blindai-preview/docs/docs/how-to-guides/fake-whisper-model.onnx
100%|██████████| 233M/233M [00:04<00:00, 56.9MB/s] 


>If you or your service provider are running your own instance of the BlindAI server and you don't wish to conenct to our managed cloud solution, you can pass your `BlindAiConnection` client object to the `transcribe` method's `connection` option.

We can print out the returned `transcript` variable to check our results are as expected.

In [ ]:
print(transcript)

[' Now go away, or I shall taunt you a second timer!']


Our audio file has been correctly transcribed. Feel free to test this out with your own audio files!

### Conclusions

This is the end of our introduction to BlindAI!

Please check out the rest of our BlindAI documentation to see more examples of how you can use BlindAI to query AI models without compromising the safety of user data or models.